In [1]:
import sys, os
import numpy as np
import glob
import pandas as pd
import math

from scipy import stats
import matplotlib.pyplot as plt
from astropy.io import fits

from bokeh.io import output_notebook, show, export_png,export_svgs, save
from bokeh.plotting import figure, show, output_file
import bokeh.palettes
from bokeh.layouts import column,gridplot
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper
from bokeh.models.glyphs import Text
from bokeh.palettes import Category20
from bokeh.transform import linear_cmap

from ics.cobraCharmer import pfi as pfiControl
from ics.cobraCharmer import pfiDesign
from astropy.table import Table


In [40]:
class ontimeModel():
    
    def getPhiFwdSlope(self, pid, dataframe):

        ndf = dataframe.loc[dataframe['fiberNo'] == pid].loc[dataframe['J2_fwd'] > 0.001]

        onTimeArray = ndf['J2onTimeFwd'].values
        angSpdArray = ndf['J2_fwd'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(onTimeArray,angSpdArray)

        # If the slope is nan, that means the linear regression is failed.  Return zero instead of nan.
        if math.isnan(slope) is True:
            slope = 0

        return slope,intercept

    def getPhiRevSlope(self, pid, dataframe):

        ndf = dataframe.loc[dataframe['fiberNo'] == pid].loc[dataframe['J2_rev'] < 0]

        onTimeArray = ndf['J2onTimeRev'].values
        angSpdArray = ndf['J2_rev'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(onTimeArray,angSpdArray)

        # If the slope is nan, that means the linear regression is failed.  Return zero instead of nan.
        if math.isnan(slope) is True:
            slope = 0

        return slope,intercept

    def buildPhiModelfromDataFrame(self, dataframe, visibleFibers=False):

        # Reading all model and build a model list

        j2fwd_slope = np.zeros(57)
        j2fwd_int = np.zeros(57)
        
        j2rev_slope = np.zeros(57)
        j2rev_int = np.zeros(57)


        for pid in visibleFibers:
            
            fw_s, fw_i=self.getPhiFwdSlope(pid,dataframe)
            rv_s, rv_i=self.getPhiRevSlope(pid,dataframe)
            
            j2fwd_slope[pid-1] = fw_s
            j2fwd_int[pid-1] = fw_i
            
            j2rev_slope[pid-1] = rv_s
            j2rev_int[pid-1] = rv_i
       
        self.j2fwd_slope = j2fwd_slope
        self.j2fwd_int   = j2fwd_int
        self.j2rev_slope = j2rev_slope
        self.j2rev_int   = j2rev_int



In [3]:
def plotJ2OntimeSpeed(GroupIdx, dataFrame, xrange, yrange):

    mapper = Category20[len(GroupIdx)]
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    p = figure( tools=TOOLS, x_range=xrange, y_range=yrange,plot_height=500, plot_width=1000)

    colorcode = 0
    for i in GroupIdx:
        legendname="Fiber "+str(i)

        p.line(x=dataFrame['J2onTimeFwd'][dataFrame['fiberNo'] == i], y=dataFrame['J2_fwd'][dataFrame['fiberNo'] == i],\
        color=mapper[colorcode],line_width=2,legend=legendname)
        p.circle(x=dataFrame['J2onTimeFwd'][dataFrame['fiberNo'] == i], y=dataFrame['J2_fwd'][dataFrame['fiberNo'] == i],radius=0.1,\
            color=mapper[colorcode],fill_color=None)
        p.line(x=dataFrame['J2onTimeRev'][dataFrame['fiberNo'] == i], y=dataFrame['J2_rev'][dataFrame['fiberNo'] == i],color=mapper[colorcode],line_width=2)
        p.circle(x=dataFrame['J2onTimeRev'][dataFrame['fiberNo'] == i], y=dataFrame['J2_rev'][dataFrame['fiberNo'] == i],radius=0.1,\
            color=mapper[colorcode],fill_color=None)

        colorcode = colorcode + 1

    return p


In [4]:
path='/Volumes/GoogleDrive/My Drive/PFS/CobraModuleTestResult/Science15/ImageData/20190730/phi-ontime/'

In [5]:
brokens = [1,55]
visibles= [e for e in range(1,58) if e not in brokens]
goodIdx = np.array(visibles) - 1
goodGroupIdx = {}
for group in range(6):
    goodGroupIdx[group] = goodIdx[goodIdx%6==group] + 1


In [6]:
dataarray=[]
pidarray=[]
otfarray=[]
otrarray=[]
j2fwarray=[]
j2rvarray=[]

imageSet = ['run1', 'run2', 'run3', 'run4']

for i in imageSet:
    fw_file=path+f'{i}'+'/phiSpeedFW.npy'
    rv_file=path+f'{i}'+'/phiSpeedRv.npy'
    J2_fwd=np.load(fw_file)*180.0/math.pi 
    J2_rev=-np.load(rv_file)*180.0/math.pi
    
    xml = glob.glob(path+f'{i}'+'/*.xml')[0]
    model = pfiDesign.PFIDesign(xml)
    
    J2onTimeFwd = np.zeros(len(J2_fwd[goodIdx]))+model.motorOntimeSlowFwd2[goodIdx]*1000
    J2onTimeRev = np.zeros(len(J2_rev[goodIdx]))+model.motorOntimeSlowRev2[goodIdx]*1000
    pid=np.array(visibles)
    pidarray.append(pid)
    otfarray.append(J2onTimeFwd)
    otrarray.append(J2onTimeRev)
    
    j2fwarray.append(J2_fwd[goodIdx])
    j2rvarray.append(J2_rev[goodIdx])
    
d={'fiberNo': np.array(pidarray).flatten(),
   'J2onTimeFwd':  np.array(otfarray).flatten(), 'J2onTimeRev':  np.array(otrarray).flatten(),
    'J2_fwd': np.array(j2fwarray).flatten(), 'J2_rev': np.array(j2rvarray).flatten()}
df = pd.DataFrame(d)


In [ ]:
d

In [ ]:

TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

phirange = [10,20,30,40]

p1 = plotJ2OntimeSpeed(goodGroupIdx[0], df, [phirange[0], phirange[-1]], [-1,1])
p2 = plotJ2OntimeSpeed(goodGroupIdx[1], df, [phirange[0], phirange[-1]], [-1,1])
p3 = plotJ2OntimeSpeed(goodGroupIdx[2], df, [phirange[0], phirange[-1]], [-1,1])
p4 = plotJ2OntimeSpeed(goodGroupIdx[3], df, [phirange[0], phirange[-1]], [-1,1])
p5 = plotJ2OntimeSpeed(goodGroupIdx[4], df, [phirange[0], phirange[-1]], [-1,1])
p6 = plotJ2OntimeSpeed(goodGroupIdx[5], df, [phirange[0], phirange[-1]], [-1,1])

#x = np.array([])
#y1 = np.array([])
#y2 = np.array([])
#for tms in phirange:
    #print(np.mean(df['J2_fwd'][df['onTime']==tms].values))
#    x=np.append(x,tms)
#    y1=np.append(y1,np.median(df['J2_fwd'][df['J2onTime']==tms].values))
#    y2=np.append(y2,np.median(df['J2_rev'][df['J2onTime']==tms].values))

#slope, intercept, r_value, p_value, std_err = stats.linregress(x[:8],y1[:8])
#fit=np.polyfit(x, y*100, 1)
#print(fit)

#q = figure(tools=TOOLS, x_range=[phirange[0]-10, phirange[-1]+10], y_range=[-1.5,1.5],plot_height=500, plot_width=1000)
#q.circle(x=df['J2onTime'], y=df['J2_fwd'],radius=0.3,\
#        color='red',fill_color=None)
#q.circle(x=x,y=y1, radius=0.5, color='blue')
#legendtext=f'Y={slope:.8f}X{intercept:.2f}'
#q.line(x=x[:8], y=x[:8]*slope+intercept,color='blue',line_width=3, legend=legendtext)

#slope, intercept, r_value, p_value, std_err = stats.linregress(x[:8],y2[:8])
#fit=np.polyfit(x, y*100, 1)
#q.circle(x=df['J2onTime'], y=df['J2_rev'],radius=0.3,\
#        color='green',fill_color=None)
#q.circle(x=x,y=y2, radius=0.5, color='#3B0F6F')
#legendtext=f'Y={slope:.8f}X+{intercept:.2f}'
#q.line(x=x[:8], y=x[:8]*slope+intercept,color='#3B0F6F',line_width=3, legend=legendtext)

output_file("phi.html")
save(column(p1,p2,p3,p4,p5,p6), filename="phi.html",title='Phi On-time')


In [42]:
otm = ontimeModel()
otm.buildPhiModelfromDataFrame(df, visibleFibers=visibles)

fwd_target = np.zeros(len(otm.j2fwd_int))+0.075
rev_target = np.zeros(len(otm.j2fwd_int))-0.075
newOntimeSlowFwd2 = (fwd_target-otm.j2fwd_int)/otm.j2fwd_slope
newOntimeSlowRev2 = (rev_target-otm.j2rev_int)/otm.j2rev_slope

newOntimeFwd2 = ((3.0*fwd_target)-otm.j2fwd_int)/otm.j2fwd_slope
newOntimeRev2 = ((3.0*rev_target)-otm.j2rev_int)/otm.j2rev_slope


ind = np.where(newOntimeFwd2 > 80)
newOntimeFwd2[ind] = 80

ind = np.where(newOntimeRev2 > 80)
newOntimeRev2[ind] = 80


#newOntimeRev2[18]=60
#newOntimeFwd2 = otm.getTargetOnTime(0.07,otm.j2fwd_slope, model.motorOntimeFwd2 ,j2fwd_avg)
#newOntimeRev2 = otm.getTargetOnTime(-0.07,otm.j2rev_slope, model.motorOntimeRev2 ,j2rev_avg)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
56
57


/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [44]:
otm.j2fwd_int

array([ 0.        , -0.0433988 , -0.17148214, -0.03896916, -0.04644228,
       -0.04362485, -0.03134307, -0.09156482, -0.04737614, -0.12273999,
       -0.07483309, -0.09599503, -0.03544106, -0.04318941, -0.04489369,
       -0.03596415, -0.10413995, -0.05917015, -0.10767235, -0.04110776,
       -0.04401784, -0.03637029, -0.07740159, -0.05514218, -0.03726653,
       -0.07639382, -0.0306133 , -0.03140808, -0.04617502, -0.02567104,
       -0.04328229, -0.03299096, -0.08337725, -0.10555057, -0.13841623,
       -0.03766071, -0.03115528, -0.05261221, -0.03273543, -0.10977697,
       -0.05114718, -0.10986519, -0.03926388, -0.07365443, -0.03158449,
       -0.02167136, -0.02521508, -0.08462458, -0.04381696, -0.04127367,
       -0.11752036, -0.03665207, -0.02589751, -0.02830212,  0.        ,
       -0.09036763, -0.03498056])

In [45]:
newOntimeSlowFwd2, newOntimeSlowRev2

(array([        inf, 16.62654193, 20.50157934, 16.57352187, 16.66242508,
        17.75211576, 24.03327843, 22.01991781, 23.8234997 , 21.17699564,
        12.72816974, 14.71236092, 15.42702585, 17.98786147, 16.12165536,
        12.4278437 , 22.03864579, 15.20825905, 21.04662113, 18.09791263,
        13.97369378, 16.59632687, 15.55045502, 26.06955688, 11.87674739,
        25.99073717, 13.42895563, 15.97577054, 15.94841262, 17.90827367,
        19.49621104, 18.15538855, 31.38691845, 13.30265771, 20.28325326,
        17.4406736 , 13.15987438, 18.54901094, 20.91010032, 20.18289684,
        17.50134689, 24.19856606, 12.43008851, 13.92398764, 16.67167244,
        21.12436436, 31.42842755, 11.67032494, 25.74262921, 18.77848836,
        18.68906673, 18.02869437, 16.55829621, 17.12742755,         inf,
        25.87556925, 17.97185028]),
 array([       -inf, 16.30796649, 18.98313575, 17.25635441, 15.25873231,
        20.34275536, 37.00979846, 26.99634744, 24.56696655, 21.3824241 ,
        11.5524

In [13]:
newOntimeFwd2, newOntimeRev2

(array([37.69078682, 32.97808955, 38.38668854, 37.24309762, 40.19949892,
        57.93291711, 41.84996153, 53.02465779, 37.24126941, 25.47051818,
        27.61831574, 36.37986681, 40.81714303, 36.29159315, 29.22765236,
        40.49235695, 32.21083955, 38.32889263, 41.47866659, 31.58495374,
        38.94922666, 30.85586133, 56.11695334, 27.74534301, 51.74218837,
        32.50177505, 38.49629248, 35.69061533, 44.59162891, 44.22038215,
        43.3733171 , 61.11364889, 24.35440176, 34.53937624, 40.66173288,
        31.75509726, 40.3521882 , 50.02322491, 36.56716055, 38.31197541,
        43.83333241, 28.74769673, 27.97401092, 40.13428859, 53.90195883,
        78.46989426, 22.6369868 , 58.24131012, 43.00386266, 33.25043557,
        42.24951034, 41.17480579, 41.99733433, 49.34651947, 42.48324958]),
 array([36.6543751 , 30.74244299, 41.3511419 , 33.73230966, 45.40802737,
        80.        , 54.74602279, 56.21447745, 39.56073294, 27.79327348,
        30.72536448, 41.63608672, 42.13925235, 32

In [ ]:
newOntimeFwd2

In [49]:
initXML = '/Volumes/GoogleDrive/My Drive/PFS/CobraModuleTestResult/Science15/XML/Science15_converted.xml'
newXML = '/Volumes/GoogleDrive/My Drive/PFS/CobraModuleTestResult/Science15/XML/Science15_phiopt_20190731.xml'
#pfi = pfiControl.PFI(fpgaHost='localhost', doConnect=False) #'fpga' for real device.
#pfi.loadModel(initXML)

model=pfiDesign.PFIDesign(initXML)
phiTable = 'phiOntime.csv'

OntimeFwd2=model.motorOntimeFwd2
OntimeRev2=model.motorOntimeRev2

SlowOntimeFwd2=model.motorOntimeSlowFwd2
SlowOntimeRev2=model.motorOntimeSlowRev2

OntimeFwd2[goodIdx] = newOntimeFwd2[goodIdx]/1000.0
OntimeRev2[goodIdx] = newOntimeRev2[goodIdx]/1000.0

SlowOntimeFwd2[goodIdx] = newOntimeSlowFwd2[goodIdx]/1000.0
SlowOntimeRev2[goodIdx] = newOntimeSlowRev2[goodIdx]/1000.0

pid=range(len(OntimeFwd2))
j2fwd_int = np.zeros(len(OntimeFwd2))
j2fwd_int[goodIdx] = otm.j2fwd_int[goodIdx]

j2fwd_slope = np.zeros(len(OntimeFwd2))
j2fwd_slope[goodIdx] = otm.j2fwd_slope[goodIdx]


j2rev_int = np.zeros(len(OntimeRev2))
j2rev_int[goodIdx] = otm.j2rev_int[goodIdx]
j2rev_slope = np.zeros(len(OntimeRev2))
j2rev_slope[goodIdx] = otm.j2rev_slope[goodIdx]



t=Table([pid, model.motorOntimeFwd2, j2fwd_int, j2fwd_slope, SlowOntimeFwd2,
         model.motorOntimeRev2,j2rev_int, j2rev_slope, SlowOntimeRev2],
         names=('Fiber No','Ori Fwd OT', 'FWD int', 'FWD slope', 'New Fwd OT',
                'Ori Rev OT', 'REV int', 'REV slope', 'New Rev OT'),
         dtype=('i2','f4', 'f4', 'f4','f4', 'f4', 'f4', 'f4', 'f4'))
t.write(phiTable,format='ascii.ecsv',overwrite=True, 
        formats={'Fiber No':'%i','Ori Fwd OT': '%10.5f', 'FWD int': '%10.5f', 'FWD slope': '%10.5f', 'New Fwd OT': '%10.5f',\
        'Ori Rev OT': '%10.5f', 'REV int': '%10.5f', 'REV slope': '%10.5f', 'New Rev OT': '%10.5f'})



model.updateOntimes(phiFwd=SlowOntimeFwd2, phiRev=SlowOntimeRev2, fast=False)
model.updateOntimes(phiFwd=OntimeFwd2, phiRev=OntimeRev2, fast=True)

model.createCalibrationFile(newXML)

In [43]:
len(otm.j2fwd_int)# , , otm.j2fwd_slope, SlowOntimeFwd2,
        # model.motorOntimeRev2,otm.j2rev_int, otm.j2rev_slope, SlowOntimeRev2)

57

In [ ]:
arm='phi'
repeat = 3
    
for tms in tarray:
    for n in range(repeat):
        phi_list = glob.glob(path+f'{tms:0>4d}/{arm}*{n}.fits.gz')
        #print(path+f'{tms:0>4d}/{arm}*{n}.fits.gz')
        a = 1000.0
            
        fig=plt.figure(figsize=(10, 10))
        columns = 1
        rows = len(phi_list)

        ax = []
        i = 0
        for f in phi_list:
            hdu = fits.open(f)
            maxi = 0.5*np.max(hdu[0].data)
            image = np.log10(a*(hdu[0].data/maxi)+1)/np.log10(a)
            #image = hdu[0].data
            basename = os.path.basename(f)

            xsize = [0,2500]
            ysize = [700, 1100]


            ax.append( fig.add_subplot(rows, columns, i+1) )
            if i < len(phi_list)-1:
                ax[-1].get_xaxis().set_ticks([])
            ax[-1].set_title(basename, fontsize = 10)
            plt.imshow(image[ysize[0]:ysize[1],xsize[0]:xsize[1]],cmap='gray')
            i=i+1

        plt.savefig(path+f'{tms:0>4d}/{arm}{tms}Stacked{n}.png')

In [ ]:
%matplotlib qt
fitsfile='/Volumes/Disk/Data/20190530/phi/0015/phi1ForwardStack0.fits.gz'
hdu = fits.open(fitsfile)
image = np.log10(a*(hdu[0].data/maxi)+1)/np.log10(a)
xsize = [0,2500]
ysize = [700, 1100]
plt.imshow(image[ysize[0]:ysize[1],xsize[0]:xsize[1]],cmap='gray')
plt.show()

In [ ]:
dd=df.loc[df['fiberNo'] == 32]
dd
x = range(10,50)
y_predicted = [otm.j2fwd_slope[31]*i + otm.j2fwd_int[31]  for i in x]

In [ ]:
newOntimeFwd2

In [ ]:
p=figure()
p.circle(x=newOntimeFwd2[31],y=[0.07],color='red')
p.circle(x=dd['J2onTime'],y=dd['J2_fwd'])
p.line(x,y_predicted,color='red')
show(p)

In [33]:
xml='/Volumes/GoogleDrive/My Drive/PFS/CobraModuleTestResult/Science15/XML/Science15_converted.xml'
calibModel = pfiDesign.PFIDesign(xml)
speed_fwd = np.load('/Volumes/GoogleDrive/My Drive/PFS/CobraData/20190610/phiSpeedFW_A.npy')
speed_rev = np.load('/Volumes/GoogleDrive/My Drive/PFS/CobraData/20190610/phiSpeedRV_A.npy')

In [ ]:
calibModel.motorOntimeSlowFwd2

In [ ]:
speed_fwd

In [68]:
def plotBestPhiOnTimeFwd(DataFrame, OntimeModel, fiberNo):
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    title_string=f"Fiber {fiberNo+1} Phi Fwd"
    p = figure(tools=TOOLS, x_range=[0,50], y_range=[0,0.4],
               plot_height=400, plot_width=500,title=title_string)
    p.xaxis.axis_label = 'On Time'
    p.yaxis.axis_label = 'Speed'
    dd=df.loc[df['fiberNo'] == fiberNo+1]

    newOntimeFwd2 = (0.075-otm.j2fwd_int[fiberNo])/otm.j2fwd_slope[fiberNo]
    print(newOntimeFwd2)

    x = range(10,50)
    y_predicted = [otm.j2fwd_slope[fiberNo]*i + otm.j2fwd_int[fiberNo]  for i in x]
    p.circle(x=newOntimeFwd2,y=[0.075],color='red', fill_color=None, radius=1.0)
    #if calibModel is not None:
    #    p.circle(x=calibModel.motorOntimeSlowFwd2[fiberNo-1]*1000.0,y=speed_fwd[fiberNo-1]*180/math.pi,
    #             color='blue', fill_color=None, radius=1.0)

    p.circle(x=dd['J2onTimeFwd'],y=dd['J2_fwd'])
    p.line(x,y_predicted,color='red')
    
    return p

def plotBestPhiOnTimeRev(DataFrame, OntimeModel, calibModel, speed_rev, fiberNo):
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    title_string=f"Fiber {fiberNo} Phi Rev"
    p = figure(tools=TOOLS, x_range=[0,50], y_range=[-0.4,0],
               plot_height=400, plot_width=500,title=title_string)
    p.xaxis.axis_label = 'On Time'
    p.yaxis.axis_label = 'Speed'
    dd=df.loc[df['fiberNo'] == fiberNo]

    newOntimeRev2 = (-0.075-otm.j2rev_int[fiberNo-1])/otm.j2rev_slope[fiberNo-1]


    x = range(10,50)
    y_predicted = [otm.j2rev_slope[fiberNo-1]*i + otm.j2rev_int[fiberNo-1]  for i in x]
    p.circle(x=newOntimeRev2,y=[-0.07],color='red', fill_color=None, radius=1.0)
    p.circle(x=calibModel.motorOntimeSlowRev2[fiberNo-1]*1000.0,y=-speed_rev[fiberNo-1]*180/math.pi,
             color='blue', fill_color=None, radius=1.0)

    p.circle(x=dd['J2onTimeRev'],y=dd['J2_rev'])
    p.line(x,y_predicted,color='red')
    
    return p




In [69]:
parray=[]

for f in goodIdx:
    p=plotBestPhiOnTimeFwd(df, otm,  f)
    parray.append(p)

grid = gridplot(parray, ncols=3, plot_width=400, plot_height=400)
show(grid)

16.6265419255057
20.5015793423024
16.57352187055105
16.662425084618192
17.752115761815897
24.033278430603254
22.019917807214316
23.823499696195444
21.17699563761112
12.728169742404173
14.712360915714127
15.427025849912594
17.987861470919693
16.12165536156938
12.427843700353748
22.03864578911963
15.208259050738398
21.046621132795767
18.09791263415628
13.973693780762293
16.59632686886517
15.550455018531329
26.069556882458166
11.876747394520065
25.99073717378375
13.428955633254038
15.975770542642836
15.948412615513464
17.908273674912333
19.496211035705674
18.15538854937521
31.386918451403492
13.30265771199681
20.283253261306225
17.44067359783159
13.15987438469563
18.549010942779073
20.91010032058129
20.182896836801444
17.501346889717663
24.198566056101466
12.43008850527386
13.923987643313128
16.671672440506164
21.124364356101978
31.42842754993359
11.670324941821496
25.742629207943914
18.778488355377934
18.689066725638757
18.028694374884534
16.558296206375374
17.127427550061757
25.87556924

In [ ]:
parray=[]

for f in goodIdx:
    p=plotBestPhiOnTimeRev(df, otm, calibModel, speed_rev,  f+1)
    parray.append(p)

grid = gridplot(parray, ncols=3, plot_width=400, plot_height=400)
show(grid)

In [65]:
SlowOntimeFwd2

array([0.025332  , 0.01662654, 0.02050158, 0.01657352, 0.01666243,
       0.01775212, 0.02403328, 0.02201992, 0.0238235 , 0.021177  ,
       0.01272817, 0.01471236, 0.01542703, 0.01798786, 0.01612166,
       0.01242784, 0.02203865, 0.01520826, 0.02104662, 0.01809791,
       0.01397369, 0.01659633, 0.01555046, 0.02606956, 0.01187675,
       0.02599074, 0.01342896, 0.01597577, 0.01594841, 0.01790827,
       0.01949621, 0.01815539, 0.03138692, 0.01330266, 0.02028325,
       0.01744067, 0.01315987, 0.01854901, 0.0209101 , 0.0201829 ,
       0.01750135, 0.02419857, 0.01243009, 0.01392399, 0.01667167,
       0.02112436, 0.03142843, 0.01167032, 0.02574263, 0.01877849,
       0.01868907, 0.01802869, 0.0165583 , 0.01712743, 0.021285  ,
       0.02587557, 0.01797185])